In [1]:
import elasticsearch
from elasticsearch import Elasticsearch
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import statistics
es= Elasticsearch([{'host': '10.10.6.90','port': 9200}])
es.ping()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


False

In [ ]:
try:
    res = es.search(index="slide_picking_from_scanner", doc_type="", body={"_source":{}}, size=1000000,)
    print("slide_picking_from_scanner ACQUIRED SUCCESSFULLY")
    slide_picking_from_scanner = pd.json_normalize(res['hits']['hits'])
    slide_picking_from_scanner = slide_picking_from_scanner.sort_values(by = ['_source.data.row_index'], ascending = True)
    slide_picking_from_scanner = slide_picking_from_scanner.sort_values(by = ['_source.data.col_index'], ascending = True)
    slide_picking_from_scanner['row_col'] = (slide_picking_from_scanner['_source.data.row_index']+1).astype(str)+"_"+(slide_picking_from_scanner['_source.data.col_index']+1).astype(str)
    slide_picking_from_scanner['_source.data.date'] = pd.to_datetime(slide_picking_from_scanner['_source.data.time_stamp']).dt.date

    
    res1 = es.search(index="slide_placement", doc_type="", body={"_source":{}}, size=1000000,)
    print("slide_placement ACQUIRED SUCCESSFULLY")
    slide_placement = pd.json_normalize(res1['hits']['hits'])
    slide_placement = slide_placement.sort_values(by = ['_source.data.row_index'], ascending = True)
    slide_placement = slide_placement.sort_values(by = ['_source.data.col_index'], ascending = True)
    slide_placement['row_col'] = (slide_placement['_source.data.row_index']+1).astype(str)+"_"+(slide_placement['_source.data.col_index']+1).astype(str)
    slide_placement['_source.data.date'] = pd.to_datetime(slide_placement['_source.data.time_stamp']).dt.date

    
    
    res2 = es.search(index="slide_locking", doc_type="", body={"_source":{}}, size=1000000,)
    print("slide_locking ACQUIRED SUCCESSFULLY")
    slide_locking = pd.json_normalize(res2['hits']['hits'])
    slide_locking = slide_locking.sort_values(by = ['_source.data.row_index'], ascending = True)
    slide_locking = slide_locking.sort_values(by = ['_source.data.col_index'], ascending = True)
    slide_locking['_source.data.date'] = pd.to_datetime(slide_locking['_source.data.time_stamp']).dt.date
    slide_locking['row_col'] = (slide_locking['_source.data.row_index']+1).astype(str)+"_"+(slide_locking['_source.data.col_index']+1).astype(str)

    
    
    res3 = es.search(index="inline_corrections", doc_type="", body={"_source":{}}, size=1000000,)
    print("inline_corrections ACQUIRED SUCCESSFULLY")
    inline_corrections = pd.json_normalize(res3['hits']['hits'])
    inline_corrections = inline_corrections.sort_values(by = ['_source.data.row_index'], ascending = True)
    inline_corrections = inline_corrections.sort_values(by = ['_source.data.col_index'], ascending = True)
    inline_corrections['_source.data.date'] = pd.to_datetime(inline_corrections['_source.data.time_stamp']).dt.date
    inline_corrections['row_col'] = (inline_corrections['_source.data.row_index']+1).astype(str)+"_"+(inline_corrections['_source.data.col_index']+1).astype(str)

    a = inline_corrections['_source.data.scanner_name'].unique()
    print("Scanners available are:",a)
    scanner = input("Please enter the scanner of your choice = ")
    lst = a
    if scanner in lst:
        d = inline_corrections[inline_corrections['_source.data.scanner_name']==scanner]
        d = d.sort_values(by = ['_source.data.date'], ascending = True)
        z = d.drop_duplicates(subset=['_source.data.load_identifier'], keep='last')
        loadd = z['_source.data.load_identifier'].iloc[-1]
#         loadd = 'CS003-3-1649307411699'
        x4 = d[d['_source.data.load_identifier'] == loadd]
        x4 = x4.drop_duplicates(subset=['_source.data.slide_id'])
        x4['_source.data.computed_angledegree'] = x4['_source.data.computed_angle']*57.2958
        x4 = x4.sort_values(by = ['_source.data.row_index','_source.data.col_index'], ascending = True)

        a = slide_placement[slide_placement['_source.data.scanner_name']==scanner]
        a = a.sort_values(by = ['_source.data.date'], ascending = True)
        x1 = a[a['_source.data.load_identifier'] == loadd]
        x1 = x1.drop_duplicates(subset=['_source.data.slide_id'])
        x1 = x1.sort_values(by = ['_source.data.row_index','_source.data.col_index'], ascending = True)

        b = slide_picking_from_scanner[slide_picking_from_scanner['_source.data.scanner_name']==scanner]
        b = b.sort_values(by = ['_source.data.date'], ascending = True)
        x2 = b[b['_source.data.load_identifier'] == loadd]
        x2 = x2.drop_duplicates(subset=['_source.data.slide_id'])
        x2 = x2.sort_values(by = ['_source.data.row_index','_source.data.col_index'], ascending = True)

        c = slide_locking[slide_locking['_source.data.scanner_name']==scanner]
        c = c.sort_values(by = ['_source.data.date'], ascending = True)
        x3 = c[c['_source.data.load_identifier'] == loadd]
        x3 = x3.drop_duplicates(subset=['_source.data.slide_id'])
        x3 = x3.sort_values(by = ['_source.data.row_index','_source.data.col_index'], ascending = True)

        fig = make_subplots(rows=4, cols=2,
            subplot_titles=("Slide Width", "Slide Height", "Inline Computed Angle", "Slide Angle While Placing"
                            , "X Offset", "Y Offset", "Slide Locking Status", "Slide Angle While Picking"))

        #PLOTTING
        fig.add_trace(go.Scatter(y=x1["row_col"], x=x1["_source.data.offset_pos_x_um"],mode="markers"),
                        row=3, col=1)
        fig.add_trace(go.Scatter(x=x1["row_col"], y=x1["_source.data.offset_pos_y_um"],mode="markers"),
                        row=3, col=2)
        fig.add_trace(go.Scatter(x=x1["row_col"], y=x1["_source.data.slide_height_um"],mode="markers"),
                        row=1, col=2)
        fig.add_trace(go.Scatter(y=x1["row_col"], x=x1["_source.data.slide_width_um"],mode="markers"),
                        row=1, col=1)
        fig.add_trace(go.Scatter(x=x1["row_col"], y=x1["_source.data.actual_angle"],mode="markers"),
                        row=2, col=2)
        fig.add_trace(go.Scatter(x=x2["row_col"], y=x2["_source.data.actual_angle"],mode="markers"),
                        row=4, col=2)
        fig.add_trace(go.Bar(x=x3["row_col"], y=x3["_source.data.first_current_diff"]),
                        row=4, col=1)
        fig.add_trace(go.Bar(x=x3["row_col"], y=x3["_source.data.second_current_diff"]),
                        row=4, col=1)
        fig.add_trace(go.Scatter(x=x4["row_col"], y=x4["_source.data.computed_angledegree"],mode="markers"),
                        row=2, col=1)


        fig.update_yaxes(type = "category",row = 1, col = 1)
        fig.update_xaxes(type = "category",row = 1, col = 2)
        fig.update_xaxes(type = "category",row = 2, col = 1)
        fig.update_xaxes(type = "category",row = 2, col = 2)
        fig.update_yaxes(type = "category",row = 3, col = 1)
        fig.update_xaxes(type = "category",row = 3, col = 2)
        fig.update_xaxes(type = "category",row = 4, col = 1)
        fig.update_xaxes(type = "category",row = 4, col = 2)

        #AXES TITLES
        fig.update_xaxes(title_text='Offset in microns',row = 3, col = 1)
        fig.update_yaxes(title_text='Slide Position',row = 3, col = 1)

        fig.update_yaxes(title_text='Offset in microns',row = 3, col = 2)
        fig.update_xaxes(title_text='Slide Position',row = 3, col = 2)

        fig.update_yaxes(title_text='Slide Height in microns',row = 1, col = 2)
        fig.update_xaxes(title_text='Slide Position',row = 1, col = 2)

        fig.update_xaxes(title_text='Slide Width in microns',row = 1, col = 1)
        fig.update_yaxes(title_text='Slide Position',row = 1, col = 1)

        fig.update_yaxes(title_text='Slide angle in degrees',row = 2, col = 2)
        fig.update_xaxes(title_text='Slide Position',row = 2, col = 2)

        fig.update_yaxes(title_text='Slide angle in degrees',row = 4, col = 2)
        fig.update_xaxes(title_text='Slide Position',row = 4, col = 2)

        fig.update_yaxes(title_text='Slide Locking Final Current',row = 4, col = 1)
        fig.update_xaxes(title_text='Slide Position',row = 4, col = 1)

        fig.update_yaxes(title_text='Slide Entering Angle',row = 2, col = 1)
        fig.update_xaxes(title_text='Slide Position',row = 2, col = 1)

        fig.update_xaxes(range=[-5000,5000], row = 3, col = 1)
        fig.update_yaxes(range=[-1500,6500], row = 3, col = 2)
        fig.update_yaxes(range=[72000,78000], row = 1, col = 2)
        fig.update_xaxes(range=[22000,28000], row = 1, col = 1)
        fig.update_yaxes(range=[-5,5], row = 2, col = 2)
        fig.update_yaxes(range=[-5,5], row = 4, col = 2)
        fig.update_yaxes(range=[-100,500], row = 4, col = 1)
        fig.update_yaxes(range=[-7,7], row = 2, col = 1)

        #BOUNDARY LINES
        fig.add_vline(x=3500, line_dash="dot", row=3, col=1, line_color="red", line_width=2)
        fig.add_vline(x=-3500, line_dash="dot", row=3, col=1, line_color="red", line_width=2)

        fig.add_hline(y=0, line_dash="dot", row=3, col=2, line_color="red", line_width=2)
        fig.add_hline(y=5000, line_dash="dot", row=3, col=2, line_color="red", line_width=2)

        fig.add_hline(y=73500, line_dash="dot", row=1, col=2, line_color="red", line_width=2)
        fig.add_hline(y=76500, line_dash="dot", row=1, col=2, line_color="red", line_width=2)

        fig.add_vline(x=23500, line_dash="dot", row=1, col=1, line_color="red", line_width=2)
        fig.add_vline(x=26500, line_dash="dot", row=1, col=1, line_color="red", line_width=2)

        fig.add_hline(y=-2, line_dash="dot", row=2, col=2, line_color="red", line_width=2)
        fig.add_hline(y=2, line_dash="dot", row=2, col=2, line_color="red", line_width=2)

        fig.add_hline(y=-1.5, line_dash="dot", row=4, col=2, line_color="red", line_width=2)
        fig.add_hline(y=1.5, line_dash="dot", row=4, col=2, line_color="red", line_width=2)

        fig.add_hline(y=150, line_dash="dot", row=4, col=1, line_color="red", line_width=2)

        fig.add_hline(y=-4, line_dash="dot", row=2, col=1, line_color="red", line_width=2)
        fig.add_hline(y=4, line_dash="dot", row=2, col=1, line_color="red", line_width=2)

        #OUTLIERS
        a11 = x1[x1["_source.data.offset_pos_x_um"]<-3500]
        a12 = x1[x1["_source.data.offset_pos_x_um"]>3500]
        fig.add_trace(go.Scatter(y=a11["row_col"], x=a11["_source.data.offset_pos_x_um"],mode="markers", marker = dict(color = "red")),row = 3, col = 1)
        fig.add_trace(go.Scatter(y=a12["row_col"], x=a11["_source.data.offset_pos_x_um"],mode="markers", marker = dict(color = "red")),row = 3, col = 1)

        a21 = x1[x1["_source.data.offset_pos_y_um"]<0]
        a22 = x1[x1["_source.data.offset_pos_y_um"]>5000]
        fig.add_trace(go.Scatter(x=a21["row_col"], y=a21["_source.data.offset_pos_y_um"],mode="markers", marker = dict(color = "red")),row = 3, col = 2)
        fig.add_trace(go.Scatter(x=a22["row_col"], y=a22["_source.data.offset_pos_y_um"],mode="markers", marker = dict(color = "red")),row = 3, col = 2)

        a31 = x1[x1["_source.data.slide_height_um"]<73500]
        a32 = x1[x1["_source.data.slide_height_um"]>76500]
        fig.add_trace(go.Scatter(x=a31["row_col"], y=a31["_source.data.slide_height_um"],mode="markers", marker = dict(color = "red")),row = 1, col = 2)
        fig.add_trace(go.Scatter(x=a32["row_col"], y=a32["_source.data.slide_height_um"],mode="markers", marker = dict(color = "red")),row = 1, col = 2)

        a41 = x1[x1["_source.data.slide_width_um"]<23500]
        a42 = x1[x1["_source.data.slide_width_um"]>26500]
        fig.add_trace(go.Scatter(y=a41["row_col"], x=a41["_source.data.slide_width_um"],mode="markers", marker = dict(color = "red")),row = 1, col = 1)
        fig.add_trace(go.Scatter(y=a42["row_col"], x=a42["_source.data.slide_width_um"],mode="markers", marker = dict(color = "red")),row = 1, col = 1)

        a51 = x1[x1["_source.data.actual_angle"]<-2]
        a52 = x1[x1["_source.data.actual_angle"]>2]
        fig.add_trace(go.Scatter(x=a51["row_col"], y=a51["_source.data.actual_angle"],mode="markers", marker = dict(color = "red")),row = 2, col = 2)
        fig.add_trace(go.Scatter(x=a52["row_col"], y=a52["_source.data.actual_angle"],mode="markers", marker = dict(color = "red")),row = 2, col = 2)

        a61 = x2[x2["_source.data.actual_angle"]<-1.5]
        a62 = x2[x2["_source.data.actual_angle"]>1.5]
        fig.add_trace(go.Scatter(x=a61["row_col"], y=a61["_source.data.actual_angle"],mode="markers", marker = dict(color = "red")),row = 4, col = 2)
        fig.add_trace(go.Scatter(x=a62["row_col"], y=a62["_source.data.actual_angle"],mode="markers", marker = dict(color = "red")),row = 4, col = 2)

        a71 = x3[x3["_source.data.first_current_diff"]<150]

        a81 = x4[x4["_source.data.computed_angledegree"]<-4]
        a82 = x4[x4["_source.data.computed_angledegree"]>4]
        fig.add_trace(go.Scatter(x=a81["row_col"], y=a81["_source.data.computed_angledegree"],mode="markers", marker = dict(color = "red")),row = 2, col = 1)
        fig.add_trace(go.Scatter(x=a82["row_col"], y=a82["_source.data.computed_angledegree"],mode="markers", marker = dict(color = "red")),row = 2, col = 1)

        p = len(a11) + len(a12)
        q = len(a21) + len(a22)
        r = len(a31) + len(a32)
        s = len(a41) + len(a42)
        t = len(a51) + len(a52)
        u = len(a61) + len(a62)
        v = len(a71)
        w = len(a81) + len(a82)


        #ANNOTATIONS
        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x1["_source.data.offset_pos_x_um"]))+"<br><b>Average X-Offset : "+\
                           str(round(statistics.mean(x1["_source.data.offset_pos_x_um"]),2))+"</b><br>Max X-Offset : <b>"+str(max(x1["_source.data.offset_pos_x_um"]))+\
                           "</b><br>Min X-Offset : <b>"+str(min(x1["_source.data.offset_pos_x_um"]))+"</b><br>Skewness : <b>"+str(round(x1["_source.data.offset_pos_x_um"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x1["_source.data.offset_pos_x_um"]),2))
                           +"</b><br>Reject Count : <b>"+str(p)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=3,col=1,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)


        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x1["_source.data.offset_pos_y_um"]))+"<br><b>Average Y-Offset : "+\
                           str(round(statistics.mean(x1["_source.data.offset_pos_y_um"]),2))+"</b><br>Max Y-Offset : <b>"+str(max(x1["_source.data.offset_pos_y_um"]))+\
                           "</b><br>Min Y-Offset : <b>"+str(min(x1["_source.data.offset_pos_y_um"]))+"</b><br>Skewness : <b>"+str(round(x1["_source.data.offset_pos_y_um"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x1["_source.data.offset_pos_y_um"]),2))
                           +"</b><br>Reject Count : <b>"+str(q)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=3,col=2,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)



        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x1["_source.data.slide_height_um"]))+"<br><b>Average Slide Height : "+\
                           str(round(statistics.mean(x1["_source.data.slide_height_um"]),2))+"</b><br>Max Slide Height : <b>"+str(round(max(x1["_source.data.slide_height_um"]),2))+\
                           "</b><br>Min Slide Height : <b>"+str(round(min(x1["_source.data.slide_height_um"]),2))+"</b><br>Skewness : <b>"+str(round(x1["_source.data.slide_height_um"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x1["_source.data.slide_height_um"]),2))
                           +"</b><br>Reject Count : <b>"+str(r)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=1,col=2,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)



        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x1["_source.data.slide_width_um"]))+"<br><b>Average Slide Width : "+\
                           str(round(statistics.mean(x1["_source.data.slide_width_um"]),2))+"</b><br>Max Slide Width : <b>"+str(round(max(x1["_source.data.slide_width_um"]),2))+\
                           "</b><br>Min Slide Width : <b>"+str(round(min(x1["_source.data.slide_width_um"]),2))+"</b><br>Skewness : <b>"+str(round(x1["_source.data.slide_width_um"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x1["_source.data.slide_width_um"]),2))
                           +"</b><br>Reject Count : <b>"+str(s)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=1,col=1,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)



        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x1["_source.data.actual_angle"]))+"<br><b>Average Slide Angle : "+\
                           str(round(statistics.mean(x1["_source.data.actual_angle"]),2))+"</b><br>Max Slide Angle : <b>"+str(round(max(x1["_source.data.actual_angle"]),2))+\
                           "</b><br>Min Slide Angle : <b>"+str(round(min(x1["_source.data.actual_angle"]),2))+"</b><br>Skewness : <b>"+str(round(x1["_source.data.actual_angle"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x1["_source.data.actual_angle"]),2))
                           +"</b><br>Reject Count : <b>"+str(t)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=2,col=2,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)



        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x2["_source.data.actual_angle"]))+"<br><b>Average Slide Angle : "+\
                           str(round(statistics.mean(x2["_source.data.actual_angle"]),2))+"</b><br>Max Slide Angle : <b>"+str(round(max(x2["_source.data.actual_angle"]),2))+\
                           "</b><br>Min Slide Angle : <b>"+str(round(min(x2["_source.data.actual_angle"]),2))+"</b><br>Skewness : <b>"+str(round(x2["_source.data.actual_angle"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x2["_source.data.actual_angle"]),2))
                           +"</b><br>Reject Count : <b>"+str(u)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=4,col=2,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)


        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x3["_source.data.first_current_diff"]))+"<br><b>Average Slide Locking Current Difference : "+\
                           str(round(statistics.mean(x3["_source.data.first_current_diff"]),2))+"</b><br>Max Slide Locking Current Difference: <b>"+str(round(max(x3["_source.data.first_current_diff"]),2))+\
                           "</b><br>Min Slide Locking Current Difference: <b>"+str(round(min(x3["_source.data.first_current_diff"]),2))+"</b><br>Skewness : <b>"+str(round(x3["_source.data.first_current_diff"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x3["_source.data.first_current_diff"]),2))
                           +"</b><br>Reject Count : <b>"+str(v)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=4,col=1,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)


        fig.add_annotation(text="<b>Total Number of Slides : <b>"+str(len(x4["_source.data.computed_angledegree"]))+"<br><b>Average Slide Incoming Angle : "+\
                           str(round(statistics.mean(x4["_source.data.computed_angledegree"]),2))+"</b><br>Max Slide Incoming Angle : <b>"+str(round(max(x4["_source.data.computed_angledegree"]),2))+\
                           "</b><br>Min Slide Incoming Angle : <b>"+str(round(min(x4["_source.data.computed_angledegree"]),2))+"</b><br>Skewness : <b>"+str(round(x4["_source.data.computed_angledegree"].skew(),2))
                           +"</b><br>Standard Deviation : <b>"+str(round(statistics.stdev(x4["_source.data.computed_angledegree"]),2))
                           +"</b><br>Reject Count : <b>"+str(w)
        ,showarrow=False,font=dict(family="Courier New, monospace",size=10,color="black"),row=2,col=1,
        xref="paper", yref="paper",bordercolor="#c7c7c7",borderwidth=2,borderpad=4,bgcolor="#ffffff",opacity=0.4)

        fig.update_layout(barmode = "overlay")
        fig.update_layout(height=1500, width=1200)
        fig.update_layout(showlegend=False)
        fig.update_layout(hovermode = 'x unified',title = "SCANNER STATUS: " + scanner)
        
        fig.show()

    else:
        print("No such scanner found")


except:
    print("No Data Found")
    
rejfromslideplacement = pd.concat([a11,a12,a21,a22,a31,a32,a41,a42,a51,a52])
rejfromslidepicking = pd.concat([a61,a62])
rejfromlockingstatus = pd.concat([a71])
rejfrominlinecomp = pd.concat([a81,a82])
rej = pd.concat([rejfromslideplacement, rejfromslidepicking, rejfromlockingstatus, rejfrominlinecomp])
rej = rej.drop_duplicates(subset=['_source.data.slide_id'])
print("Total Number of Rejected Slides:", len(rej))

In [ ]:
x1

In [ ]:
rej[['_source.data.slide_id','row_col','_source.data.actual_angle']]

In [ ]:
slide_placement

In [ ]:
rej.columns

In [ ]:
x2[x2['_source.data.slide_id'] == 4046]

In [ ]:
x3[x3['_source.data.slide_id'] == 4046]

In [ ]:
rejfromslideplacement = pd.concat([a11,a12,a21,a22,a31,a32,a41,a42,a51,a52])
rejfromslideplacement

In [ ]:
rejfromslidepicking = pd.concat([a61,a62])
rejfromslidepicking

In [ ]:
rejfromlockingstatus = pd.concat([a71])
rejfromlockingstatus

In [ ]:
rejfrominlinecomp = pd.concat([a81,a82])
rejfrominlinecomp